In [1]:
# ! pip install ogb
# ! python -c "import ogb; print(ogb.__version__)"
# # Otherwise, please update the version by running

# ! pip install torch_geometric
# ! pip install --upgrade torch_geometric ogb

# ! pip install matplotlib

In [ ]:
from ogb.linkproppred import PygLinkPropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data, stored in './dataset/ogbl_collab/'
dataset = PygLinkPropPredDataset(name="ogbl-collab", root='dataset/')


/root/miniconda3/envs/cs276p/lib/python3.12/site-packages/ogb/linkproppred/dataset_pyg.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.lo

In [ ]:
# Get split edges from the dataset
split_edge = dataset.get_edge_split()

# Extract edge lists for train, valid, and test splits
train_edges = split_edge["train"]['edge']  # List of training edges
valid_edges = split_edge["valid"]['edge']  # List of validation edges
test_edges = split_edge["test"]['edge']    # List of test edges

from torch_geometric.loader import Data

# Convert to Data objects if necessary
train_data = Data(edge_index=train_edges.T)  # Transpose to match PyG format
valid_data = Data(edge_index=valid_edges.T)
test_data = Data(edge_index=test_edges.T)

# Create DataLoaders
train_loader = DataLoader([train_data], batch_size=32, shuffle=True)
valid_loader = DataLoader([valid_data], batch_size=32, shuffle=False)
test_loader = DataLoader([test_data], batch_size=32, shuffle=False)


/root/miniconda3/envs/cs276p/lib/python3.12/site-packages/ogb/linkproppred/dataset_pyg.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train = replace_numpy_with_torcht

In [ ]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data
from torch.utils.data import DataLoader

# Define a simple GCN-based link prediction model
class GCNLinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNLinkPredictor, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x
    
    def predict(self, x_i, x_j):
        # Dot product to predict link probability
        return (x_i * x_j).sum(dim=-1)

# Load graph data
data = dataset[0]  # PyG graph object
in_channels = data.num_features
hidden_channels = 64
out_channels = 32

# Instantiate the model and optimizer
model = GCNLinkPredictor(in_channels, hidden_channels, out_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    x = model(data.x, data.edge_index)
    
    # Positive edges
    pos_edge_index = train_edges.T
    
    # Sample negative edges
    neg_edge_index = negative_sampling(pos_edge_index, num_nodes=data.num_nodes)
    
    # Compute link prediction scores for positive and negative edges
    pos_pred = model.predict(x[pos_edge_index[0]], x[pos_edge_index[1]])
    neg_pred = model.predict(x[neg_edge_index[0]], x[neg_edge_index[1]])
    
    # Labels for positive and negative edges
    pos_label = torch.ones(pos_pred.size(0))
    neg_label = torch.zeros(neg_pred.size(0))
    
    # Concatenate predictions and labels
    pred = torch.cat([pos_pred, neg_pred], dim=0)
    label = torch.cat([pos_label, neg_label], dim=0)
    
    # Loss
    loss = torch.nn.functional.binary_cross_entropy_with_logits(pred, label)
    loss.backward()
    optimizer.step()
    return loss.item()

# Training the model
for epoch in range(100):
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

# Evaluate on validation or test set
def evaluate(edge_index):
    model.eval()
    with torch.no_grad():
        x = model(data.x, data.edge_index)
        pos_pred = model.predict(x[edge_index[0]], x[edge_index[1]])
        return torch.sigmoid(pos_pred)

# Predict on validation and test edges
valid_scores = evaluate(valid_edges.T)
test_scores = evaluate(test_edges.T)

print("Validation Scores:", valid_scores)
print("Test Scores:", test_scores)


Epoch 0, Loss: 0.6290480494499207
Epoch 10, Loss: 0.574454665184021
Epoch 20, Loss: 0.5366992950439453
Epoch 30, Loss: 0.4889083504676819
Epoch 40, Loss: 0.47436967492103577
Epoch 50, Loss: 0.4617832899093628
Epoch 60, Loss: 0.4526648223400116
Epoch 70, Loss: 0.4492711126804352


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

# Convert to NetworkX graph for visualization
G = to_networkx(data, to_undirected=True)

# Plot original graph with nodes and edges
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G, seed=42)  # Use spring layout for visualization

# Draw original graph nodes and edges
nx.draw_networkx_nodes(G, pos, node_size=50, node_color="blue", alpha=0.7)
nx.draw_networkx_edges(G, pos, alpha=0.5)

# Visualize the predicted links with high confidence
def visualize_predicted_links(edge_index, scores, threshold=0.5):
    for i, (u, v) in enumerate(edge_index.T):
        # Check if the score exceeds the threshold
        if scores[i] > threshold:
            # Draw the edge if the score is high
            nx.draw_networkx_edges(
                G, pos, edgelist=[(u.item(), v.item())], edge_color="red", alpha=0.3, width=2
            )

# Assume `valid_scores` and `test_scores` contain predicted probabilities for links
# Visualize links with high confidence in validation and test sets
visualize_predicted_links(valid_edges.T, valid_scores, threshold=0.8)
visualize_predicted_links(test_edges.T, test_scores, threshold=0.8)

# Show plot
plt.title("Graph Visualization with Predicted Links")
plt.show()
